In [5]:
import pandas as pd
import sklearn
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [69]:
# root_mean_square_error is in sklearn 1.4.+
# !pip install --upgrade scikit-learn

     |████████████████████████████████| 13.4 MB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 67.2 MB/s eta 0:00:01
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


In [4]:
# this is needed to read parquet files... only need to do it once
# !pip install pyarrow

     |████████████████████████████████| 39.9 MB 6.1 MB/s eta 0:00:01


In [5]:
# Q1: Read the data for January. How many columns are there?
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
# df.head()
# Q1: ANSWER
len(df.columns)

19

In [6]:
#Q2: What's the standard deviation of the trips duration in January?
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

df['duration'] =  df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df['duration'].std()


42.594351241920904

In [7]:
# Q3: Next, we need to check the distribution of the duration variable. 
# There are some outliers. Let's remove them and keep only the records 
# where the duration was between 1 and 60 minutes (inclusive).

# What fraction of the records left after you dropped the outliers?
before = len(df)



In [8]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

after = len(df)

print(f"pct records left after outliers: {after/before}")


pct records left after outliers: 0.9812202822125979


In [9]:
# Q4: Let's apply one-hot encoding to the pickup and dropoff location IDs. 
# We'll use only these two features for our model.

# Turn the dataframe into a list of dictionaries 
# (remember to re-cast the ids to strings - otherwise it will label encode them)
# Fit a dictionary vectorizer
# Get a feature matrix from it
# What's the dimensionality of this matrix (number of columns)?

# define features for one-hot encoding, cast to strings
# PULocation & DOLocation are ints it will try and label
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df[categorical] = df[categorical].astype(str)


In [10]:
# we need dicts
train_dicts = df[categorical + numerical].to_dict(orient='records')

In [11]:
# dictionary vectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [12]:
# feature matrix
X_train

<3009173x516 sparse matrix of type '<class 'numpy.float64'>'
	with 9027519 stored elements in Compressed Sparse Row format>

In [ ]:
#ANSWER: 516

In [13]:
# Q5: Now let's use the feature matrix from the previous step to train a model.

# Train a plain linear regression model with default parameters, where duration is the response variable
# Calculate the RMSE of the model on the training data
# What's the RMSE on train?

# get the target values
target = 'duration'
y_train = df[target].values

In [14]:
# use linear regression to train x to Y

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649143388169879

In [ ]:
#Q6: Now let's apply this model to the validation dataset (February 2023).

#What's the RMSE on validation?

In [10]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [11]:
# prep the dataframes
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [12]:
# set up one-encoding
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [13]:
dv = DictVectorizer()

# training inputs
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# predictive inputs
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [14]:
# set the Y targets for training data and predictive data

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
# train the 
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)